In [1]:
# Get Learning Cruves

The goal of this jupyter notebook is to get the learning curves accuracies in tran, validation an test for the baseline model and MLNT-teacher and MLNT-student neural networks for each epoch.

It is intended to be used in google colab.
A folder named *learning_curves* needs to be created in order to store the results.
It retrieves data from the checkpoint folder

SyntaxError: ignored

In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 4.8MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 112kB 26.9MB/s 
     |████████████████████████████████| 460kB 28.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=571b5f835aa5ddb4ddf10b8719edb5ce1476fb5df2c1d53b9fb7ac7e2dbfc2a2
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=e6b11dab7d8c3644eae1aed2fa6bb6612019aae6db602a01508d22bc2f977b7b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import wandb
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar')

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
# from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import pandas as pd




noise file noisy_label_kv.txt generated with noise: 0.5



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [0]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model

In [5]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()

if use_cuda:
  net.cuda()


Model setup
| Building network: resnet34


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:122: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:124: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:125: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


In [0]:
def load_checkpoint(epoch):
  checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + "_" + str(epoch) + '.pth.tar')
  net.load_state_dict(checkpoint['state_dict'])

In [0]:
def test(loader):
    net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs).cuda()

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total

    return(round(float(test_acc),2))

In [0]:
def get_accy_list(max_epoch, loader):
  max_epoch = 101

  accy_list = []
  for epoch in range(1,101):
    load_checkpoint(epoch)
    accy_list.append(test(loader))
  return(accy_list)
    # print(accy_list)

In [0]:
print("computing test list\n")
test_list  = get_accy_list(config.num_epochs, test_loader)
print(test_list)
print("computing val list\n")
valid_list = get_accy_list(config.num_epochs, valid_loader)
print(valid_list)
print("computing train list\n")
train_list = get_accy_list(config.num_epochs, train_loader)

df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

computing test list



In [0]:
from IPython.display import Image
Image(filename='/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/data/test/airplane/1001.png') 

In [0]:
df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

df

In [0]:
clean_kv = config.drive_dir +"/learning_curves/"+ config.checkpoint+".csv"
# clean_kv = "/content/drive/My Drive/Colab_Notebooks/pfm/data/asdf.txt"
df.to_csv(clean_kv, sep=';', index=False,header = True)